In [1]:
"""@author: César Godoy Delaigue"""

import pandas as pd
import numpy as np
import xlrd
import pyodbc
import sqlalchemy
import unicodedata

### 1.- Cargar archivo de Resultados

In [59]:

file_path = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\01. Portezuelo\00 Campaña mayo SGS\ALS\36668_2022_FLOW HYDRO.xlsx"

# Carga del archivo Excel
excel_data = pd.ExcelFile(file_path)


In [61]:
# Leyendo el contenido de la hoja "Resultados"
results_sheet = excel_data.parse('Resultados')

# Mostrando las primeras filas de la hoja "Resultados" para entender su estructura y contenido
results_sheet.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# Explorando más filas de la hoja "Resultados" para encontrar datos útiles
results_sheet_data = results_sheet.dropna(how='all')  # Eliminando filas completamente vacías

# Mostrando algunas filas con datos para entender el contenido de la hoja
results_sheet_data.head(10)  # Mostrando las primeras 10 filas con datos


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
8,Muestras del Grupo: 36668/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,N° ALS,NaN,NaN,NaN,NaN,NaN,292048/2022-1.0,292047/2022-1.0,292049/2022-1.0
10,Fecha de Muestreo,NaN,NaN,NaN,NaN,NaN,26/05/2022,26/05/2022,26/05/2022
11,Hora de Muestreo,NaN,NaN,NaN,NaN,NaN,11:20:00,12:20:00,13:00:00
12,Tipo de Muestra,NaN,NaN,NaN,NaN,NaN,Agua Subterránea,Agua Subterránea,Agua Subterránea
13,Identificación,NaN,NaN,NaN,NaN,NaN,PMP-2 PORTEZUELO,PMP-1 PORTEZUELO,SH-01
14,Método de Análisis,Parámetro,CM,Unidad,LD,LQ,Valor,Valor,Valor
15,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Carbonato,11030,mg CaCO3/L,1,---,< 1,< 1,< 1
17,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Bicarbonato,11030,mg CaCO3/L,1,---,165,184,115


In [63]:
# Transformando los datos de formato ancho a largo usando 'melt' de pandas
# Primero, prepararemos los datos eliminando filas y columnas no necesarias y estableciendo los encabezados correctos

# Eliminamos las primeras filas que no contienen datos de medición relevantes
data_for_melting = results_sheet_data.iloc[6:].reset_index(drop=True)

# Estableciendo la primera fila como encabezado
data_for_melting.columns = data_for_melting.iloc[0]
data_for_melting = data_for_melting.iloc[1:].reset_index(drop=True)

In [64]:
# Revisando el dataframe para identificar el problema con los valores NaN
# Primero revisaremos los nombres de las columnas para asegurarnos de que no hay errores allí
data_for_melting.columns = data_for_melting.iloc[0]  # Estableciendo la primera fila como encabezados
data_for_melting = data_for_melting[1:]  # Removiendo la primera fila después de convertirla en encabezado

# Verificando si hay columnas con nombre NaN y cuántas columnas hay en total
column_nan_check = data_for_melting.columns.isnull().any(), data_for_melting.columns.size

data_for_melting


,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Carbonato,11030,mg CaCO3/L,1,---,< 1,< 1,< 1
2,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Bicarbonato,11030,mg CaCO3/L,1,---,165,184,115
3,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Total,11030,mg CaCO3/L,1,---,165,184,115
4,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Hidróxido,11030,mg CaCO3/L,1,---,< 1,< 1,< 1
5,Alcalinidades (SM 2320-A) INN/SMA,Fecha de Análisis,11030,---,---,---,01/06/2022 10:00,01/06/2022 10:00,01/06/2022 10:00
...,...,...,...,...,...,...,...,...,...
90,Metal Total ICP-MS (EPA) INN/SMA,Titanio Total,12680,mg/L,---,"0,0002","0,0008","0,0012","0,0170"
91,Metal Total ICP-MS (EPA) INN/SMA,Uranio Total,12680,mg/L,---,"0,00006","0,00353","0,00134","< 0,00006"
92,Metal Total ICP-MS (EPA) INN/SMA,Vanadio Total,12680,mg/L,---,"0,0003","0,0015","0,0014","0,0015"
93,Metal Total ICP-MS (EPA) INN/SMA,Zinc Total,12680,mg/L,---,"0,0007","0,0584","0,0614","0,0691"


In [76]:
# Antes de hacer la transformación, necesitamos asegurarnos de que los nombres de las columnas son correctos y no hay NaNs
# Vamos a configurar los nombres de las columnas manualmente para evitar problemas

column_names = [
    "Método de Análisis", 
    "Parámetro", 
    "CM", 
    "Unidad", 
    "LD", 
    "LQ", 
    "PMP-2 PORTEZUELO", 
    "PMP-1 PORTEZUELO", 
    "SH-01", 
    # "542887/2023-1.0", 
    # "542888/2023-1.0", 
    # "542889/2023-1.0", 
    # "542890/2023-1.0", 
    # "542891/2023-1.0"
]

# Asignamos los nombres de las columnas al dataframe
data_for_melting.columns = column_names

# Ahora procederemos a hacer la transformación al formato largo
melted_data = pd.melt(data_for_melting, 
                    id_vars=["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"], 
                    var_name='Sample_ID', 
                    value_name='Value')

melted_data.head()


,Método de Análisis,Parámetro,CM,Unidad,LD,LQ,Sample_ID,Value
0,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Carbonato,11030,mg CaCO3/L,1,---,PMP-2 PORTEZUELO,< 1
1,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Bicarbonato,11030,mg CaCO3/L,1,---,PMP-2 PORTEZUELO,165
2,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Total,11030,mg CaCO3/L,1,---,PMP-2 PORTEZUELO,165
3,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Hidróxido,11030,mg CaCO3/L,1,---,PMP-2 PORTEZUELO,< 1
4,Alcalinidades (SM 2320-A) INN/SMA,Fecha de Análisis,11030,---,---,---,PMP-2 PORTEZUELO,01/06/2022 10:00


In [79]:

# Cargamos nuevamente los datos originales de la hoja de resultados
original_data = excel_data.parse('Resultados')

# # extraer las filas con los datos de identificación, fecha y tipo de muestra
identification_data = original_data.iloc[9:14, 1:].transpose()
identification_data.columns = identification_data.iloc[0]
identification_data = identification_data.drop(identification_data.index[0])
identification_data.reset_index(drop=True, inplace=True)

# # Renombrando las columnas para facilitar el acceso
identification_data.columns = ["N° Informe_LB", "Fecha de Muestreo", "Hora de Muestreo", "Tipo de Muestra", "FLOW"]

# # Unimos los datos de identificación con los datos fundidos basándonos en el índice
combined_data = pd.merge(melted_data, identification_data, left_on='Sample_ID', right_on='FLOW')

# # Eliminamos la columna 'Sample_ID' y ajustamos el dataframe para que 'FLOW' sea el nuevo 'Sample_ID'
combined_data.drop('Sample_ID', axis=1, inplace=True)
combined_data.rename(columns={'FLOW': 'Sample_ID'}, inplace=True)

# Reorganizando las columnas para que 'Sample_ID', 'Fecha de Muestreo', y 'Tipo de Muestra' aparezcan primero
column_order = ['Sample_ID', 'Fecha de Muestreo', 'Tipo de Muestra', 'Método de Análisis', 'Parámetro', 
                'CM', 'Unidad', 'LD', 'LQ', 'N° Informe_LB', 'Value']
combined_data = combined_data[column_order]

# # Mostramos los primeros registros para verificar que los cambios se han realizado correctamente
# combined_data.head()
combined_data



,Sample_ID,Fecha de Muestreo,Tipo de Muestra,Método de Análisis,Parámetro,CM,Unidad,LD,LQ,N° Informe_LB,Value
0,PMP-2 PORTEZUELO,26/05/2022,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Carbonato,11030,mg CaCO3/L,1,---,292048/2022-1.0,< 1
1,PMP-2 PORTEZUELO,26/05/2022,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Bicarbonato,11030,mg CaCO3/L,1,---,292048/2022-1.0,165
2,PMP-2 PORTEZUELO,26/05/2022,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Total,11030,mg CaCO3/L,1,---,292048/2022-1.0,165
3,PMP-2 PORTEZUELO,26/05/2022,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Hidróxido,11030,mg CaCO3/L,1,---,292048/2022-1.0,< 1
4,PMP-2 PORTEZUELO,26/05/2022,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Fecha de Análisis,11030,---,---,---,292048/2022-1.0,01/06/2022 10:00
...,...,...,...,...,...,...,...,...,...,...,...
277,SH-01,26/05/2022,Agua Subterránea,Metal Total ICP-MS (EPA) INN/SMA,Titanio Total,12680,mg/L,---,"0,0002",292049/2022-1.0,"0,0170"
278,SH-01,26/05/2022,Agua Subterránea,Metal Total ICP-MS (EPA) INN/SMA,Uranio Total,12680,mg/L,---,"0,00006",292049/2022-1.0,"< 0,00006"
279,SH-01,26/05/2022,Agua Subterránea,Metal Total ICP-MS (EPA) INN/SMA,Vanadio Total,12680,mg/L,---,"0,0003",292049/2022-1.0,"0,0015"
280,SH-01,26/05/2022,Agua Subterránea,Metal Total ICP-MS (EPA) INN/SMA,Zinc Total,12680,mg/L,---,"0,0007",292049/2022-1.0,"0,0691"


In [80]:
# Guardaremos el dataframe transformado en un nuevo archivo Excel para revisar

output_file_path = 'Resultados_Formato1.xlsx'

# Guardando el dataframe en un archivo Excel
combined_data.to_excel(output_file_path, index=False)

output_file_path

'Resultados_Formato1.xlsx'

In [79]:
combined_data.shape

(472, 11)

In [80]:

# Eliminando las filas donde la columna 'Parámetro' es igual a 'Fecha de Análisis'
combined_data_cleaned = combined_data[combined_data['Parámetro'] != 'Fecha de Análisis']

# Mostrando las primeras filas del dataframe limpio para confirmar la limpieza
combined_data_cleaned.head()


,Sample_ID,Fecha de Muestreo,Tipo de Muestra,Método de Análisis,Parámetro,CM,Unidad,LD,LQ,N° Informe_LB,Value
0,FLOW-DAND-077,16/08/2023,Agua Subterránea,Alcalinidad,Alcalinidad Total,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"358,0"
1,FLOW-DAND-077,16/08/2023,Agua Subterránea,Alcalinidad,Bicarbonato,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"358,0"
2,FLOW-DAND-077,16/08/2023,Agua Subterránea,Alcalinidad,Carbonato,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"< 1,0"
3,FLOW-DAND-077,16/08/2023,Agua Subterránea,Alcalinidad,Hidróxido,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"< 1,0"
5,FLOW-DAND-077,16/08/2023,Agua Subterránea,Aniones (Cromatografía Iónica),Fluoruro,26051,mg/L,"0,01","0,06",542893/2023-1.1,"< 0,06"


In [81]:
combined_data_cleaned.dtypes

Sample_ID             object
Fecha de Muestreo     object
Tipo de Muestra       object
Método de Análisis    object
Parámetro             object
CM                    object
Unidad                object
LD                    object
LQ                    object
N° Informe_LB         object
Value                 object
dtype: object

In [82]:
combined_data_cleaned.shape

(416, 11)

In [83]:
combined_data_cleaned['Fecha de Muestreo'] = pd.to_datetime(combined_data_cleaned['Fecha de Muestreo'])


C:\Users\cgodo\AppData\Local\Temp\ipykernel_26608\3113897565.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  combined_data_cleaned['Fecha de Muestreo'] = pd.to_datetime(combined_data_cleaned['Fecha de Muestreo'])
C:\Users\cgodo\AppData\Local\Temp\ipykernel_26608\3113897565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data_cleaned['Fecha de Muestreo'] = pd.to_datetime(combined_data_cleaned['Fecha de Muestreo'])


In [84]:
combined_data_cleaned.dtypes

Sample_ID                     object
Fecha de Muestreo     datetime64[ns]
Tipo de Muestra               object
Método de Análisis            object
Parámetro                     object
CM                            object
Unidad                        object
LD                            object
LQ                            object
N° Informe_LB                 object
Value                         object
dtype: object

In [85]:
combined_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 470
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Sample_ID           416 non-null    object        
 1   Fecha de Muestreo   416 non-null    datetime64[ns]
 2   Tipo de Muestra     416 non-null    object        
 3   Método de Análisis  416 non-null    object        
 4   Parámetro           416 non-null    object        
 5   CM                  416 non-null    object        
 6   Unidad              416 non-null    object        
 7   LD                  416 non-null    object        
 8   LQ                  416 non-null    object        
 9   N° Informe_LB       416 non-null    object        
 10  Value               416 non-null    object        
dtypes: datetime64[ns](1), object(10)
memory usage: 39.0+ KB


In [86]:
# Renombrando las columnas del dataframe 'combined_data_cleaned' según las especificaciones

new_column_names = {
    'Sample_ID': 'SampleID',
    'Fecha de Muestreo': 'Fecha_de_Muestreo',
    'Tipo de Muestra': 'Tipo_de_Muestra',
    'Método de Análisis': 'Metodo_de_Analisis',
    'Parámetro': 'Parametro',
    'CM': 'CM',
    'Unidad': 'Unidad',
    'LD': 'LD',
    'LQ': 'LQ',
    'N° Informe_LB': 'ID_CERTIFICADO_LAB',
    'Value': 'Valor'
}

# Aplicando el cambio de nombres de columnas
combined_data_renamed = combined_data_cleaned.rename(columns=new_column_names)

# Mostrando las primeras filas del dataframe con los nombres de columnas actualizados
combined_data_renamed.head()


,SampleID,Fecha_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor
0,FLOW-DAND-077,2023-08-16,Agua Subterránea,Alcalinidad,Alcalinidad Total,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"358,0"
1,FLOW-DAND-077,2023-08-16,Agua Subterránea,Alcalinidad,Bicarbonato,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"358,0"
2,FLOW-DAND-077,2023-08-16,Agua Subterránea,Alcalinidad,Carbonato,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"< 1,0"
3,FLOW-DAND-077,2023-08-16,Agua Subterránea,Alcalinidad,Hidróxido,26039,mg CaCO3/L,"1,0","2,5",542893/2023-1.1,"< 1,0"
5,FLOW-DAND-077,2023-08-16,Agua Subterránea,Aniones (Cromatografía Iónica),Fluoruro,26051,mg/L,"0,01","0,06",542893/2023-1.1,"< 0,06"


In [87]:

combined_data_renamed['Valor'] = combined_data_renamed['Valor'].astype(str).str.replace(',', '.')

combined_data_renamed['Unidad'] = combined_data_renamed['Unidad'].astype(str).str.replace('---', '')

combined_data_renamed['LD'] = combined_data_renamed['LD'].astype(str).str.replace('---', '')

combined_data_renamed['LQ'] = combined_data_renamed['LQ'].astype(str).str.replace('---', '')




#verificar los cambios
print(combined_data_renamed.head())


        SampleID Fecha_de_Muestreo   Tipo_de_Muestra  \
0  FLOW-DAND-077        2023-08-16  Agua Subterránea   
1  FLOW-DAND-077        2023-08-16  Agua Subterránea   
2  FLOW-DAND-077        2023-08-16  Agua Subterránea   
3  FLOW-DAND-077        2023-08-16  Agua Subterránea   
5  FLOW-DAND-077        2023-08-16  Agua Subterránea   

               Metodo_de_Analisis          Parametro     CM      Unidad    LD  \
0                     Alcalinidad  Alcalinidad Total  26039  mg CaCO3/L   1,0   
1                     Alcalinidad        Bicarbonato  26039  mg CaCO3/L   1,0   
2                     Alcalinidad          Carbonato  26039  mg CaCO3/L   1,0   
3                     Alcalinidad          Hidróxido  26039  mg CaCO3/L   1,0   
5  Aniones (Cromatografía Iónica)           Fluoruro  26051        mg/L  0,01   

     LQ ID_CERTIFICADO_LAB   Valor  
0   2,5    542893/2023-1.1   358.0  
1   2,5    542893/2023-1.1   358.0  
2   2,5    542893/2023-1.1   < 1.0  
3   2,5    542893/2023-1.1  

In [88]:
# Crear las columnas 'Qualifier' y 'Valor_BEST' a partir de la columna 'Valor'
combined_data_renamed['Qualifier'] = combined_data_renamed['Valor'].apply(lambda x: '<' if '<' in str(x) else None)
combined_data_renamed['Valor_BEST'] = combined_data_renamed['Valor'].apply(lambda x: float(str(x).replace('<', '').strip()) if '<' in str(x) else x)
combined_data_renamed['Valor_BEST_LD'] = combined_data_renamed.apply(lambda row: row['Valor_BEST'] / 2 if row['Qualifier'] == '<' else row['Valor_BEST'], axis=1)

In [89]:
# Reemplazar los espacios en la columna 'Parametro' con guiones bajos (_)
combined_data_renamed['Parametro'] = combined_data_renamed['Parametro'].apply(lambda x: x.replace(' ', '_'))

In [90]:
combined_data_renamed.dtypes

SampleID                      object
Fecha_de_Muestreo     datetime64[ns]
Tipo_de_Muestra               object
Metodo_de_Analisis            object
Parametro                     object
CM                            object
Unidad                        object
LD                            object
LQ                            object
ID_CERTIFICADO_LAB            object
Valor                         object
Qualifier                     object
Valor_BEST                    object
Valor_BEST_LD                 object
dtype: object

In [91]:
combined_data_renamed['Valor_BEST'] = combined_data_renamed['Valor_BEST'].astype('float64')
combined_data_renamed['Valor_BEST_LD'] = combined_data_renamed['Valor_BEST_LD'].astype('float64')

In [92]:
combined_data_renamed.dtypes

SampleID                      object
Fecha_de_Muestreo     datetime64[ns]
Tipo_de_Muestra               object
Metodo_de_Analisis            object
Parametro                     object
CM                            object
Unidad                        object
LD                            object
LQ                            object
ID_CERTIFICADO_LAB            object
Valor                         object
Qualifier                     object
Valor_BEST                   float64
Valor_BEST_LD                float64
dtype: object

In [93]:
valores_unicos = combined_data_renamed['Parametro'].unique()

# Convertir el array de valores únicos a un DataFrame
df_valores_unicos = pd.DataFrame(valores_unicos, columns=['Parametro'])

# Escribir el DataFrame a un archivo de Excel
df_valores_unicos.to_excel("valores_unicos_revision2.xlsx", index=False)

In [94]:
combined_data_renamed['Parametro_Unidad'] = combined_data_renamed['Parametro'] + "_" + combined_data_renamed['Unidad']

In [95]:
valores_unicos = combined_data_renamed['Parametro_Unidad'].unique()

# Convertir el array de valores únicos a un DataFrame
df_valores_unicos = pd.DataFrame(valores_unicos, columns=['Parametro_Unidad'])

# Escribir el DataFrame a un archivo de Excel
combined_data_renamed.to_excel("REVISA_2.xlsx", index=False)

In [96]:
# Crea una conexión a la base de datos
server = 'FlowDataBase\MSSQL2022_FLOW'
database = 'DBase_Codelco_GAC'
username = 'sa'
password = 'Godoca1977$'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)

In [97]:
# Crea el motor para conectarse a la base de datos
engine = sqlalchemy.create_engine('mssql+pyodbc://', creator=lambda: conn)

In [98]:
# Exporta el DataFrame a la tabla "nombre_de_la_tabla" en SQL Server utilizando fast_executemany
combined_data_renamed.to_sql('GAC_Assay_Update_79137', con=engine, if_exists='replace', index=False)

138

In [40]:
def procesar_excel(file_path, sheet_name):
    # Leer el archivo Excel, especificando que los nombres de columnas están en la fila 14 (índice 13)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=14)

    # Eliminar las primeras dos filas no deseadas
    df = df.drop([0, 1])

    # Restablecer el índice
    df.reset_index(drop=True, inplace=True)

    # Renombrar las columnas correctamente
    df.columns = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"] + df.columns[6:].tolist()

    # Transformar al formato largo
    id_vars = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"]
    melted_data = pd.melt(df, id_vars=id_vars, var_name='Sample_ID', value_name='Valor')

    return melted_data


file_path = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\01. Portezuelo\00 Campaña mayo SGS\ALS\38060_2022_FLOW HIDRO.xlsx"
# file_path = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\01. Portezuelo\00 Campaña mayo SGS\ALS\36668_2022_FLOW HYDRO.xlsx"
sheet_name = 'Resultados'
melted_data = procesar_excel(file_path, sheet_name)

# Mostrar las primeras filas del DataFrame transformado
melted_data


,Método de Análisis,Parámetro,CM,Unidad,LD,LQ,Sample_ID,Valor
0,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Carbonato,11030,mg CaCO3/L,1,---,FLOW-MLP-11,< 1
1,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Bicarbonato,11030,mg CaCO3/L,1,---,FLOW-MLP-11,205
2,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Total,11030,mg CaCO3/L,1,---,FLOW-MLP-11,205
3,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad Hidróxido,11030,mg CaCO3/L,1,---,FLOW-MLP-11,< 1
4,Alcalinidades (SM 2320-A) INN/SMA,Fecha de Análisis,11030,---,---,---,FLOW-MLP-11,07/06/2022 10:00
...,...,...,...,...,...,...,...,...
183,Metal Total ICP-MS (EPA) INN/SMA,Titanio Total,12680,mg/L,---,"0,0002",FLOW-MLP-10,"0,0128"
184,Metal Total ICP-MS (EPA) INN/SMA,Uranio Total,12680,mg/L,---,"0,00006",FLOW-MLP-10,"0,00019"
185,Metal Total ICP-MS (EPA) INN/SMA,Vanadio Total,12680,mg/L,---,"0,0003",FLOW-MLP-10,"0,0095"
186,Metal Total ICP-MS (EPA) INN/SMA,Zinc Total,12680,mg/L,---,"0,0007",FLOW-MLP-10,"0,0719"


In [58]:
from pathlib import Path
import re

def extract_path_segment(file_path, start_segment):
    # Convertir la ruta del archivo a un objeto Path
    path = Path(file_path)
    
    # Obtener las partes de la ruta
    path_parts = path.parts
    
    # Encontrar el índice del segmento de inicio
    try:
        start_index = path_parts.index(start_segment) + 1
    except ValueError:
        return "Segmento no encontrado en la ruta."
    
    # Extraer el segmento de la ruta
    extracted_path = path_parts[start_index]
    
    # Usar una expresión regular para eliminar el número y el punto iniciales
    cleaned_path = re.sub(r'^\d+\.\s*', '', extracted_path)
    
    return cleaned_path

# Ejemplo de uso
file_path = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\02. Punta Chungo\ALS"
start_segment = "Certificados"

extract_path_segment(file_path, start_segment)



'Punta Chungo'

In [46]:
import os
import pandas as pd
import re
from pathlib import Path

def procesar_excel(file_path, sheet_name):
    # Leer el archivo Excel, especificando que los nombres de columnas están en la fila 14 (índice 13)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=14)

    # Eliminar las primeras dos filas no deseadas
    df = df.drop([0, 1])

    # Restablecer el índice
    df.reset_index(drop=True, inplace=True)

    # Renombrar las columnas correctamente
    df.columns = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"] + df.columns[6:].tolist()

    # Transformar al formato largo
    id_vars = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"]
    melted_data = pd.melt(df, id_vars=id_vars, var_name='Sample_ID', value_name='Valor')

    return melted_data

def extract_path_segment(file_path, start_segment):
    # Convertir la ruta del archivo a un objeto Path
    path = Path(file_path)
    
    # Obtener las partes de la ruta
    path_parts = path.parts
    
    # Encontrar el índice del segmento de inicio
    try:
        start_index = path_parts.index(start_segment) + 1
    except ValueError:
        return "Segmento no encontrado en la ruta."
    
    # Extraer el segmento de la ruta
    extracted_path = Path(*path_parts[start_index:])
    
    # Usar una expresión regular para eliminar el número y el punto iniciales
    cleaned_path = re.sub(r'^\d+\.\s*', '', str(extracted_path))
    
    return cleaned_path

def process_directory(directory, sheet_name, start_segment):
    all_data = pd.DataFrame()
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.xlsx'):
                file_path = os.path.join(root, file)
                extracted_path = extract_path_segment(file_path, start_segment)
                try:
                    melted_data = procesar_excel(file_path, sheet_name)
                    melted_data['Ruta'] = extracted_path
                    all_data = pd.concat([all_data, melted_data], ignore_index=True)
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
    
    return all_data

# Ejemplo de uso
directory = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados"
sheet_name = 'Resultados'
start_segment = "Certificados"

all_data = process_directory(directory, sheet_name, start_segment)

# Mostrar las primeras filas del DataFrame final
print(all_data.head())


Error processing file G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\01. Portezuelo\00 Campaña mayo SGS\ALS\~$36668_2022_FLOW HYDRO.xlsx: [Errno 13] Permission denied: 'G:\\.shortcut-targets-by-id\\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\\09. Contrato CS115 EIA 2024\\03.WIP\\03.Hidroquímica\\4. Muestras FLOW\\Certificados\\01. Portezuelo\\00 Campaña mayo SGS\\ALS\\~$36668_2022_FLOW HYDRO.xlsx'
Error processing file G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\01. Portezuelo\00 Campaña mayo SGS\ALS\~$38060_2022_FLOW HIDRO.xlsx: [Errno 13] Permission denied: 'G:\\.shortcut-targets-by-id\\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\\09. Contrato CS115 EIA 2024\\03.WIP\\03.Hidroquímica\\4. Muestras FLOW\\Certificados\\01. Portezuelo\\00 Campaña mayo SGS\\ALS\\~$38060_2022_FLOW HIDRO.xlsx'
Error processing file G:\.sh